# Gerar Silver

In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pyspark.pandas as ps
import seaborn as sns

from matplotlib import dates
from pyspark.sql.functions import col, to_date
from pyspark.sql.avro.functions import *
#
# Nome da aplicação Spark
#
APP_NAME="GerarBronze"
ps.options.display.max_rows = 10

/opt/conda/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
%run StartSpark.ipynb

In [3]:
%config SqlMagic.lazy_execution = True

In [4]:
%sql spark

In [5]:
schema = """
    {
        "type": "record",
        "name": "Stock",
        "fields": [
            {"name": "ticker", "type": "string"},
            {"name": "timestamp", "type": "string"},
            {"name": "open", "type": "double"},
            {"name": "high", "type": "double"},
            {"name": "low", "type": "double"},
            {"name": "close", "type": "double"},
            {"name": "volume", "type": "long"}
        ]
    }
"""

# FK

In [21]:
from urllib.parse import urlparse

def delta_exists(delta_path, tier, table):
    url = urlparse(delta_path)

    match url:
        case "abfss":
            print("Blob Storage")
        case "s3":
        case "s3a":
            print("S3 Compatible")
    

In [23]:
from urllib.parse import urlparse

def delta_exists(delta_path, tier, table):
  """
  Checks if the provided delta path points to a supported storage type.

  Args:
      delta_path (str): The URL or path to the delta data.
      tier (str): Optional tier information (may not be used).
      table (str): Optional table name (may not be used).

  Returns:
      str: A string indicating the storage type ("Blob Storage" or "S3 Compatible")
          or None if the storage type is not supported.
  """

  url = urlparse(delta_path)
  match url.scheme:
      case "abfss":
          return "Blob Storage"
      case "s3" | "s3a":  # Combine S3 and S3A cases for efficiency
          return "S3 Compatible"
      case _:
          return None  # Return None for unsupported schemes

  # Unreachable code, but included for clarity
  # return None  # Redundant return statement here

In [24]:
delta_exists("s3a://nemesys-demo1/lakehouse", "bronze", "stocks_intraday")

'S3 Compatible'

In [25]:
delta_exists("abfss://nemesys-demo1/lakehouse", "bronze", "stocks_intraday")

'Blob Storage'